In [0]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import warnings
warnings.filterwarnings("ignore")

plt.ion()

In [0]:
class FaceLandmarkDataset(Dataset):
    
    def __init__(self, csv_file, github_url, transform=None):
        
        self.landmarks_frame = pd.read_csv(github_url+csv_file)
        self.github_url = github_url
        self.transform = transform
        
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        img_url = self.github_url+self.landmarks_frame.iloc[idx, 0]
        
        image = io.imread(img_url)
        landmarks = landmarks_frame.iloc[idx, 1:].as_matrix()
        landmarks = landmarks.astype('float').reshape(-1,2)
        sample = {'image' : image, 'landmarks': landmarks}
        
        if self.transform:
            self.transform(sample)
            
        return sample

In [0]:
class Rescale(object):
    
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size
        
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        
        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
            
        new_h, new_w = int(new_h), int(new_w)
        
        img = transform.resize(image, (new_h, new_w))
        
        landmarks = landmarks * [new_w / w, new_h / h]

        return {'image': img, 'landmarks': landmarks}

In [0]:
class RandomCrop(object):
    
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size
            
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        
        h, w = image.shape[:2]
        
        new_h, new_w = self.output_size
        
        top=np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)
        
        image = image[top: top + new_h,
                      left: left+ new_w]
        
        landmarks = landmarks - [left, top]
        
        return {'image': image, 'landmarks': landmarks}

In [0]:
class ToTensor(object):
    
    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']
        
        image = image.transpose((2,0,1))
        return {'image': torch.from_numpy(image), 
                'landmarks': torch.from_numpy(landmarks)}

In [0]:
def show_landmarks(image, landmarks):
    plt.imshow(image)
    
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='*', color='r')
    plt.pause(0.001)
    

In [0]:
github_url = 'https://raw.githubusercontent.com/CameronTaylorFL/PyTorch/master/faces/' 
csv_file = 'face_landmarks.csv'

data_set = FaceLandmarkDataset(csv_file=csv_file, github_url=github_url)

scale = Rescale(256)
crop = RandomCrop(128)

composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])


fig = plt.figure()
sample = data_set[65]

for i, tsfrm in enumerate([scale, crop, composed]):
    transformed_sample = tsfrm(sample)
    
    ax = plt.subplot(1, 4, i+1)
    plt.tight_layout()
    ax.set_title(type(tsfrm).__name__)
    show_landmarks(**transformed_sample)

show_landmarks(**sample)
plt.show()
